In [ ]:
import matplotlib.pyplot as plt
import json
import numpy as np

# Plot Accuracy and Loss 

Given correct use of the callback functions this will give a relatively simple way to plot the validation accuracy and loss of any experiment run.

Requires data on the json form: {'server_accuracy': [...], 'server_loss': [...]}.

In [ ]:
EXPERIMENT_NAME = 'CIFAR-100'
BURN_IN = 0

# Include paths to the log files that you wish to include on the structure NAME:PATH
data_paths = {
    'PA': './data/logs/cifar_pa_23_3.json',
    'AVG': './data/logs/cifar_avg_23_3.json',
    'BE': './data/logs/cifar_be_23_3.json',
    'KP': './data/logs/cifar_kp_23_3.json',
}

In [ ]:
def read_json(path):
    with open(path) as json_file:
        return json.load(json_file)

def moving_average(data, length = 10):
    moving_average_data = []
    for i, x in enumerate(data):
        if i < length:
            moving_average_data.append(sum(data[0:(i+1)]) / (i + 1))
        else:
            moving_average_data.append(sum(data[(i+1-length):(i+1)]) / length)
    return moving_average_data

In [ ]:
# Read all data and assign a color to each data
data = dict()
colors = ['red', 'blue', 'orange', 'green', 'black']
for i, key in enumerate(data_paths):
    data[key] = read_json(data_paths[key])
    data[key]['color'] = colors[i]

In [ ]:
fig = plt.figure(figsize=(14, 10))
alpha = 0.1 # Opacity of data points

for key in data:
    acc_data = data[key]['server_accuracy']
    average_data = moving_average(acc_data)
    
    plt.plot(np.arange(len(acc_data)), acc_data, marker = 'x', alpha = alpha, color = data[key]['color'])
    plt.plot(np.arange(len(average_data)), average_data, label = key, color = data[key]['color'])
    
plt.vlines(0, plt.ylim()[0], plt.ylim()[1], label = 'Burn-In', linestyles = 'dashed')
plt.legend()
plt.ylabel('Accuracy', fontsize = 20)
plt.xlabel('Round', fontsize = 20)
plt.title(EXPERIMENT_NAME + ' Accuracy', fontsize = 30)
plt.show()

In [ ]:
fig = plt.figure(figsize=(14, 10))
for key in data:
    loss_data = data[key]['server_loss']
    average_data = moving_average(loss_data)
    
    plt.plot(np.arange(len(loss_data)), loss_data, marker = 'x', alpha = alpha, color = data[key]['color'])
    plt.plot(np.arange(len(average_data)), average_data, label = key, color = data[key]['color'])
    
plt.vlines(0, plt.ylim()[0], plt.ylim()[1], label = 'Burn-In', linestyles = 'dashed')
plt.legend()
plt.ylabel('Loss', fontsize = 20)
plt.xlabel('Round', fontsize = 20)
plt.title(EXPERIMENT_NAME + ' Loss', fontsize = 30)
plt.show()

# Illustrate Client Sampling With Dirichlet Distribution

For some datasets (mainly EMNIST-62) we wish to study how algorithms perform when clients are sampled non uniformly but rather some clients are more likely than others. To do this we have to option to sample client according to a probability densisty described by a dirichlet distribution such that pmf = np.random.dirichelt([alpha] * length). To explore just how frequent different clients are sampled you can visualize that below.

In [ ]:
# Set parameters so that they are identical to your experiment.
ALPHA = 1
TOTAL_NUMBER_OF_CLIENTS = 500
CLIENTS_PER_ROUND = 20
ROUNDS = 1000

# Set the random seed for the sampling, this is the default seed in our code.
np.random.seed(1234)

client_index = np.arange(TOTAL_NUMBER_OF_CLIENTS)
samples = dict()

# If alpha is infinity we get a uniform distribution
if alpha == 'inf':
    probabilities = [1/TOTAL_NUMBER_OF_CLIENTS] * TOTAL_NUMBER_OF_CLIENTS
else:
    probabilities = np.random.dirichlet([ALPHA] * TOTAL_NUMBER_OF_CLIENTS)
    
index_total = []
for i in range(ROUNDS):
    index = np.random.choice(
            client_index,
            size = CLIENTS_PER_ROUND,
            replace = False,
            p = probabilities,
            )
    index_total.extend(index)

In [ ]:
fig = plt.figure(figsize=(10, 10))
plt.hist(index_total, bins = TOTAL_NUMBER_OF_CLIENTS)
plt.xlabel('Client', fontsize = 20)
plt.ylabel('Frequency', fontsize = 20)
plt.title('Client Frequency', fontsize = 30)
plt.show()